In [3]:
import sys
import os
current_dir = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(current_dir)))
from Grover.grover_num_list_cirq import find_num, find_num_list
from Backend.fake_backend import FakeBackend
from Backend.backend import Backend
from Backend.ibm_backend import IBM_backend

from Result.result import ResultData

from qiskit.visualization import plot_histogram, plot_coupling_map
from backend_operation import run_simulator, get_transpiled_circuits_of_circuit, get_fake_backend_list,get_unitary, get_fake_backend, get_fake_backend_list
from qiskit import transpile
import qiskit.providers.fake_provider
from qiskit_ibm_provider import IBMProvider
from qiskit.transpiler.passes import RemoveBarriers

from itertools import permutations

from SaveData.save_data_to_file import CSVWriter
import time

In [4]:
draw_circuit = False

In [36]:
# Sample of Finding a number in a list Grover circuit
winner_list = [4]
num_array = [0, 5, 7, 8, 2, 4, 9, 10, 3, 2, 9, 0, 2, 3, 4, 5]
find_num_in_data = find_num_list()
find_num_in_data.build_iteration(winner_list=winner_list, num_array=num_array, block_diagram=False, default_value=0)
find_num_in_data.create_grover(num_solutions=1, prep_value=[], block_diagram=False)
cur_experiments = find_num_in_data.measure_qc
print(f"Number of Grover circuits : {len(cur_experiments)}")
print(f"Circuit Depth is : {cur_experiments[0].depth()}")
cur_experiments[0].draw('mpl') if draw_circuit else None

Number of Grover circuits : 1
Circuit Depth is : 356


In [37]:
# Sample of Find Number Grover Circuit
find_num_in_world_obj =  find_num()
winner_list = [6]
num_array = []
find_num_in_world_obj.build_iteration(winner_num_list=[6], circuit_nqubits=3, block_diagram=False)
find_num_in_world_obj.create_grover(num_solutions=1, prep_value=[], block_diagram=False)
cur_experiments = find_num_in_world_obj.measure_qc
print(f"Number of Grover circuits : {len(cur_experiments)}")
print(f"Circuit Depth is : {cur_experiments[0].depth()}")
cur_experiments[0].draw('mpl') if draw_circuit else None

Number of Grover circuits : 1
Circuit Depth is : 26


In [ ]:
# Getting the Fake backend List
FakeBackend.get_ibm_fake_backend_name_list()

In [ ]:
# Getting the Fake backend List with qubit limits
FakeBackend.get_ibm_fake_backend_names_with_limit(20, 30)

In [14]:
# Get Fake Backend by name
FakeBackend.get_ibm_fake_backend('fake_auckland')

In [65]:
aer_backend = Backend()

In [20]:
# Using qiskit Transpile with optimaztion level 0 = No opt
transpiled_circuit_aer_opt_0 = transpile(cur_experiments[0], aer_backend.get_backend(), optimization_level=0)
print(f"Trasnpiled circuit depth : {transpiled_circuit_aer_opt_0.depth()}")
transpiled_circuit_aer_opt_0.draw('mpl') if draw_circuit else None

Trasnpiled circuit depth : 62


In [21]:
# Using qiskit Transpile with optimaztion level 3 = Max opt
transpiled_circuit_aer_opt_3 = transpile(cur_experiments[0], aer_backend.get_backend(), optimization_level=3)
print(f"Trasnpiled circuit depth : {transpiled_circuit_aer_opt_3.depth()}")
transpiled_circuit_aer_opt_3.draw('mpl') if draw_circuit else None

Trasnpiled circuit depth : 50


In [ ]:
aer_job = aer_backend.run(transpiled_circuit_aer_opt_0)
aer_res = ResultData(aer_job)
print(aer_res.get_result_time_taken())
plot_histogram(aer_res.get_counts())

In [ ]:
aer_job = aer_backend.run(transpiled_circuit_aer_opt_3)
aer_res = ResultData(aer_job)
print(aer_res.get_result_time_taken())
plot_histogram(aer_res.get_counts())

In [25]:
generic_backend = Backend(num_qubits=len(cur_experiments[0].qubits))

In [26]:
# Using qiskit Transpile with optimaztion level 0 = No opt
transpiled_circuit_generic_opt_0 = transpile(cur_experiments[0], generic_backend.get_backend(), optimization_level=0)
print(f"Trasnpiled circuit depth : {transpiled_circuit_generic_opt_0.depth()}")
transpiled_circuit_generic_opt_0.draw('mpl') if draw_circuit else None

Trasnpiled circuit depth : 62


In [27]:
# Using qiskit Transpile with optimaztion level 3 = Max opt
transpiled_circuit_generic_opt_3 = transpile(cur_experiments[0], generic_backend.get_backend(), optimization_level=3)
print(f"Trasnpiled circuit depth : {transpiled_circuit_generic_opt_3.depth()}")
transpiled_circuit_generic_opt_3.draw('mpl') if draw_circuit else None

Trasnpiled circuit depth : 59


In [ ]:
generic_job = generic_backend.run(transpiled_circuit_generic_opt_0)
generic_res = ResultData(generic_job)
print(generic_res.get_result_time_taken())
plot_histogram(generic_res.get_counts())

In [ ]:
generic_job = generic_backend.run(transpiled_circuit_generic_opt_3)
generic_res = ResultData(generic_job)
print(generic_res.get_result_time_taken())
plot_histogram(generic_res.get_counts())

In [28]:
fake_backend_name = 'fake_auckland'
fake_backend = FakeBackend(fake_backend_name)

In [29]:
# Using qiskit Transpile with optimaztion level 0 = No opt
transpiled_circuit_fake_opt_0 = transpile(cur_experiments[0], fake_backend.get_backend(), optimization_level=0)
print(f"Trasnpiled circuit depth : {transpiled_circuit_fake_opt_0.depth()}")
transpiled_circuit_fake_opt_0.draw('mpl') if draw_circuit else None

Trasnpiled circuit depth : 130


In [31]:
# Using qiskit Transpile with optimaztion level 3 = Max opt
transpiled_circuit_fake_opt_3 = transpile(cur_experiments[0], fake_backend.get_backend(), optimization_level=3)
print(f"Trasnpiled circuit depth : {transpiled_circuit_fake_opt_3.depth()}")
transpiled_circuit_fake_opt_3.draw('mpl') if draw_circuit else None

Trasnpiled circuit depth : 97


In [ ]:
fake_job = fake_backend.run(transpiled_circuit_fake_opt_0)
fake_res = ResultData(fake_job)
print(fake_res.get_result_time_taken())
plot_histogram(fake_res.get_counts())

In [ ]:
fake_job = generic_backend.run(transpiled_circuit_fake_opt_3)
fake_res = ResultData(fake_job)
print(fake_res.get_result_time_taken())
plot_histogram(fake_res.get_counts())

In [97]:
find_num_obj = find_num()
num_qubits = 6
experiments = []
seed = 42

aer_backend = Backend()
generic_backend = Backend(num_qubits=num_qubits)
fake_backend = FakeBackend('fake_auckland')
input_search_val = []
for value in range(2**num_qubits):
    if value != 6:
        continue
    input_search_val.append(value)
    find_num_obj.build_iteration(winner_num_list=[value], circuit_nqubits=num_qubits, block_diagram=False)
    find_num_obj.create_grover(num_solutions=1, prep_value=[], block_diagram=False)
    experiments.append(find_num_obj.measure_qc[0])

In [94]:
qc_without_barriar_list = [RemoveBarriers()(qc) for qc in experiments]

In [95]:
position_layout_addition = [] # list(permutations(range(num_qubits), num_qubits))

In [96]:
length = 10
seed = 42
total_start_time = time.time()

# Running on all circuits without barriars
for input_val, cur_qc in zip(input_search_val, qc_without_barriar_list):
    input_dict = {'input_search' : input_val}
    # Running on all backends
    position_layout = [None] + list(permutations(range(cur_qc.num_qubits), cur_qc.num_qubits)) + position_layout_addition
    perm_index = 0
    # Running on all layouts
    # Note : Each length, will save the file to excell
    while perm_index < len(position_layout):
        qc_transpiled_list = []
        cur_perm_list = position_layout[perm_index : perm_index + length]
        print(f"Running from index {perm_index} to {perm_index + length} of position layout\n")
        for cur_backend in [aer_backend, generic_backend, fake_backend]:
            for index, layout in enumerate(cur_perm_list):
                # Running on all opt levels
                for opt_level in [0,3]:
                    start_time = time.time()
                    qc_transpiled_list.append(cur_backend.transpile_save_param(qc=cur_qc, optimization_level=opt_level, initial_layout=layout, seed_transpiler=None, 
                                                                            **input_dict))
                    print(f"Finished circuit with Opt {opt_level} for layout {layout} indexed {index} for current loop for backend {cur_backend.get_backend().name} in {round(time.time() - start_time, 3)} Seconds")
        # Running circuits with seeds
        run_start_time = time.time()
        qc_run_result = []
        data_save = []
        for index, qc in enumerate(qc_transpiled_list):
            for seed_val in [None, seed]:
                start_time = time.time()
                qc_result = qc.backend.run(qc.transpiled_qc, shots = 1024, seed_simulator = seed_val)
                qc_result = ResultData(qc_result)
                qc_result.set_seed(seed_val)
                data_save.append([qc, qc_result])
                print(f"Finished running circuit indexed {index} out of {len(qc_transpiled_list)} with seed {seed_val} in {round(time.time() - start_time,4)}  Seconds")
        print(f"Finished running all circuits in {round(time.time() - run_start_time,4)}  Seconds")
        # Saving file for each circuit
        start_time = time.time()
        additional_txt = ""
        for key, value in input_dict.items():
            additional_txt = additional_txt + f"_K{key}_V{value}"
        excel_name = f"grover_results{additional_txt}.csv"
        save = CSVWriter(file_name=excel_name)
        save.save_data(data_save)
        print(f"Finished saving to csv at {round(time.time() - start_time,4)} seconds\n")
        perm_index = perm_index + length
print(f"### Finished all circuits in all configurations in {time.time() - total_start_time} Seconds ###")

Running from index 0 to 10 of position layout

Finished circuit with Opt 0 for layout None indexed 0 for current loop for backend aer_simulator in 0.069 Seconds
Finished circuit with Opt 3 for layout None indexed 0 for current loop for backend aer_simulator in 0.267 Seconds
Finished circuit with Opt 0 for layout (0, 1, 2, 3, 4, 5) indexed 1 for current loop for backend aer_simulator in 0.07 Seconds
Finished circuit with Opt 3 for layout (0, 1, 2, 3, 4, 5) indexed 1 for current loop for backend aer_simulator in 0.274 Seconds
Finished circuit with Opt 0 for layout (0, 1, 2, 3, 5, 4) indexed 2 for current loop for backend aer_simulator in 0.071 Seconds
Finished circuit with Opt 3 for layout (0, 1, 2, 3, 5, 4) indexed 2 for current loop for backend aer_simulator in 0.27 Seconds
Finished circuit with Opt 0 for layout (0, 1, 2, 4, 3, 5) indexed 3 for current loop for backend aer_simulator in 0.07 Seconds
Finished circuit with Opt 3 for layout (0, 1, 2, 4, 3, 5) indexed 3 for current loop for

KeyboardInterrupt: 

In [17]:
position_layout = list(permutations(range(fake_backend.get_backend_qubits()), num_qubits))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001E098B37C10>>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 801, in _clean_thread_parent_frames
    for identity in list(thread_to_parent.keys()):
                         ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


In [ ]:
length = 1
perm_index = 0
position_layout = list(permutations(range(num_qubits), num_qubits)) + position_layout_addition
while perm_index < len(position_layout):
    cur_perm = position_layout[perm_index : perm_index + length + 1]
    print(f"{cur_perm}\n")
    
    perm_index = perm_index + length + 1
    

In [8]:
length = 1
perm_index = 0
position_layout_addition = []
position_layout = list(permutations(range(num_qubits), num_qubits)) + position_layout_addition

while perm_index < len(position_layout):
    start = time.time()
    print(f"Running from index {perm_index} to {perm_index + length} of position layout\n")
    cur_perm = position_layout[perm_index : perm_index + length]
    print(f"Running {len(cur_perm)} Position layouts\n")
    # Add the Generic Aer Full Connectivity
    qc_transpiled_list = [
        generic_backend.transpile_save_param(qc=qc, optimization_level=0, initial_layout=None, seed_transpiler=None, input_search = index)
        for index, qc in enumerate(qc_find_num_list_without_barriar)
    ]
    cur_time = time.time() - start
    print(f"Opt 0 Fully Connected finished at {int(cur_time)} seconds\n")
    qc_transpiled_list = qc_transpiled_list + [
        generic_backend.transpile_save_param(qc=qc, optimization_level=3, initial_layout=None, seed_transpiler=None, input_search = index)
        for index, qc in enumerate(qc_find_num_list_without_barriar)
    ]
    cur_time = time.time() - start
    print(f"Opt 3 Fully Connected finished at {int(cur_time)} seconds\n")
    # Add the fake backend
    for layout in [None] + cur_perm:
        qc_transpiled_list = qc_transpiled_list + [
            fake_backend.transpile_save_param(qc=qc, optimization_level=0, initial_layout=layout, seed_transpiler=None, input_search = index)
            for index, qc in enumerate(qc_find_num_list_without_barriar)
        ]
        cur_time = time.time() - start
        print(f"Opt 0 FakeAucland with layout {layout} finished at {int(cur_time)} seconds\n")
        qc_transpiled_list = qc_transpiled_list + [
            fake_backend.transpile_save_param(qc=qc, optimization_level=3, initial_layout=layout, seed_transpiler=None, input_search = index)
            for index, qc in enumerate(qc_find_num_list_without_barriar)
        ]
        cur_time = time.time() - start
        print(f"Opt 3 FakeAucland with layout {layout} finished at {int(cur_time)} seconds\n")
    qc_res  = [
        qc_param.backend.run(qc_param.transpiled_qc,shots=4096)
        for qc_param in qc_transpiled_list
    ]
    cur_time = time.time() - start
    print(f"Results without seed finished at {int(cur_time)} seconds\n")
    qc_res_seed  = [
        qc_param.backend.run(qc_param.transpiled_qc,shots=4096, seed_simulator=seed)
        for qc_param in qc_transpiled_list
    ]
    cur_time = time.time() - start
    print(f"Results with seed {seed} finished at {int(cur_time)} seconds\n")
    
    qc_resultData = [ResultData(res) for res in qc_res]
    qc_resultData_seed = [ResultData(res) for res in qc_res_seed]
    [res.set_seed(seed) for res in qc_resultData_seed]
    print(f"Start saving to csv seed starting to csv\n")
    excel_name = f"grover_results_{perm_index}_to_{perm_index + length}_perm.csv"
    save = CSVWriter(file_name=excel_name)
    data = []
    for qc_transpile, res_data, res_data_seed in zip(qc_transpiled_list, qc_resultData, qc_resultData_seed):
        data.append([qc_transpile, res_data])  
        data.append([qc_transpile, res_data_seed])  
    save.save_data(data)
    cur_time = time.time() - start
    print(f"Finished saving to csv at {int(cur_time)} seconds\n")
    perm_index = perm_index + length

Running from index 0 to 1 of position layout

Running 1 Position layouts

Opt 0 Fully Connected finished at 4 seconds

Opt 3 Fully Connected finished at 25 seconds

Opt 0 FakeAucland with layout None finished at 59 seconds

Opt 3 FakeAucland with layout None finished at 240 seconds

Opt 0 FakeAucland with layout (0, 1, 2, 3, 4, 5) finished at 281 seconds

Opt 3 FakeAucland with layout (0, 1, 2, 3, 4, 5) finished at 406 seconds

Results without seed finished at 428 seconds

Results with seed 42 finished at 428 seconds

Start saving to csv seed starting to csv

Data saved to ..\logs\grover_results_0_to_1_perm_2024_08_18_23_51_39.csv
Finished saving to csv at 698 seconds

Running from index 1 to 2 of position layout

Running 1 Position layouts

Opt 0 Fully Connected finished at 5 seconds



KeyboardInterrupt: 

In [16]:
# Add the Generic Aer Full Connectivity
qc_transpiled_list = [
    generic_backend.transpile_save_param(qc=qc, optimization_level=0, initial_layout=None, seed_transpiler=None, input_search = index)
    for index, qc in enumerate(qc_find_num_list_without_barriar)
]
qc_transpiled_list = qc_transpiled_list + [
    generic_backend.transpile_save_param(qc=qc, optimization_level=3, initial_layout=None, seed_transpiler=None, input_search = index)
    for index, qc in enumerate(qc_find_num_list_without_barriar)
]
position_layout = list(position_layout)
# Add the fake backend
for layout in [None] + position_layout + position_layout_addition:
    qc_transpiled_list = qc_transpiled_list + [
        fake_backend.transpile_save_param(qc=qc, optimization_level=0, initial_layout=layout, seed_transpiler=None, input_search = index)
        for index, qc in enumerate(qc_find_num_list_without_barriar)
    ]
    qc_transpiled_list = qc_transpiled_list + [
        fake_backend.transpile_save_param(qc=qc, optimization_level=3, initial_layout=layout, seed_transpiler=None, input_search = index)
        for index, qc in enumerate(qc_find_num_list_without_barriar)
    ]

In [ ]:
# Sample that we entered an input value to the result parameters to save
qc_transpiled_list[60].backend_name

'fake_auckland'

In [ ]:
qc_res  = [
    qc_param.backend.run(qc_param.transpiled_qc,shots=1024)
    for qc_param in qc_transpiled_list
]

In [ ]:
qc_res_seed  = [
    qc_param.backend.run(qc_param.transpiled_qc,shots=1024, seed_simulator=seed)
    for qc_param in qc_transpiled_list
]

In [ ]:
len(qc_res)

224

In [ ]:
qc_resultData = [ResultData(res) for res in qc_res]

In [ ]:
qc_resultData_seed = [ResultData(res) for res in qc_res_seed]
x = [res.set_seed(seed) for res in qc_resultData_seed]

In [ ]:
len(qc_resultData)

224

In [ ]:
# excel_path = "../logs"
excel_name = "grover_results.csv"

save = CSVWriter(file_name=excel_name)

data = []
for qc_transpile, res_data, res_data_seed in zip(qc_transpiled_list, qc_resultData, qc_resultData_seed):
    data.append([qc_transpile, res_data])  
    data.append([qc_transpile, res_data_seed])  

save.save_data(data)

Data saved to ..\logs\grover_results_2024_08_09_22_03_54.csv
